In [1]:
import unittest
from public.desyncedcompiler import *
import inspect
import ast
from astrender import astrender
from asthelpers import *
import json
import ast

ds_ops = import_desynced_ops(path="./public/desyncedexport.json")

class FlowViewer(ast.NodeVisitor):
    def __init__(self):
        self.flow = {}

    def visit_DS_Call(self, node):
        self.flow[node.frame] = {'next' : node.next, 'name': node.op} 
        return ast.NodeVisitor.generic_visit(self, node)

def getFlow(code):
    if callable(code):
        code = inspect.getsource(code)
    tree = ast.parse(code, type_comments=False)
    tree = replace_binops_with_functions(tree)
    tree = flatten_calls(tree)
    tree = convert_to_ds_call(tree)
    tree = label_frames_vars(tree)
    flow_control(tree)

    flow_visitor = FlowViewer()
    flow_visitor.visit(tree)

    return tree, flow_visitor.flow

instructions initialized


In [2]:
class TestB62Encoder(unittest.TestCase):
    def test_roundtrip(self):
        ctx = b62("python")
        enced = ctx.encode(["fish"])
        result = ctx.decode(enced)
        self.assertEqual(["fish"], result)

    # Use https://stagegames.github.io/DesyncedJavaScriptUtils/ to generate known good encodes
    def test_known_list(self):
        ctx = b62()
        encoded_object = [1,2,3]
        known_good_encoding = 'DSCV02H47v03y'
        encoded = ctx.encode(encoded_object)
        self.assertEqual(known_good_encoding, encoded)

    def test_known_string(self):
        ctx = b62()
        encoded_object = "Hello World"
        known_good_encoding = 'DSCV1z4YyB1avKFY1q1MvnS'
        encoded = ctx.encode(encoded_object)
        self.assertEqual(known_good_encoding, encoded)

    def test_known_dict(self):
        '''
        Note: Naked dicts are encoded as blueprints, not behaviors by default.
        Manually change the "B" to a "C" to match
        '''
        ctx = b62()
        encoded_object = {"key": "value"}
        known_good_encoding = 'DSCV31Lnyk29K4e61r9XfB01xu'
        encoded = ctx.encode(encoded_object)
        self.assertEqual(known_good_encoding, encoded)


unittest.main(argv=[''], verbosity=3, exit=False)

test_known_dict (__main__.TestB62Encoder.test_known_dict)
Note: Naked dicts are encoded as blueprints, not behaviors by default. ... ERROR
test_known_list (__main__.TestB62Encoder.test_known_list) ... ERROR
test_known_string (__main__.TestB62Encoder.test_known_string) ... ERROR
test_roundtrip (__main__.TestB62Encoder.test_roundtrip) ... ok

ERROR: test_known_dict (__main__.TestB62Encoder.test_known_dict)
Note: Naked dicts are encoded as blueprints, not behaviors by default.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\EricV\AppData\Local\Temp\ipykernel_15956\3489309463.py", line 28, in test_known_dict
    ctx = b62()
          ^^^^^
  File "C:\Users\EricV\Documents\Desynced\compiler\PythonToDesyncedCompiler\public\desyncedcompiler.py", line 602, in __new__
    raise Exception("Unknown environment")
Exception: Unknown environment

ERROR: test_known_list (__main__.TestB62Encoder.test_known_list)
---------------

Initializing Mini Racer


In [3]:
def JustOneP(P1):
    P1+=2
    
def SkipP2(P1, P3):
    P1+=2

def ReversedParameters(P2, P1):
    P1+=2

def OneNamedParameter(named_parameter_one):
    pass

def MixedNaming(P1, named, P4, named2):
    pass
            
class TestParameterNaming(unittest.TestCase):
    def get_parameter_block(self, code):
        if callable(code):
            code = inspect.getsource(code)
        tree = ast.parse(code, type_comments=False)
        tree = replace_binops_with_functions(tree)
        tree = flatten_calls(tree)
        tree = convert_to_ds_call(tree)
        tree = label_frames_vars(tree)
        flow_control(tree)
        dso = create_dso_from_ast(tree, debug=True)
        return dso["parameters"]
        
    def test_just_p1(self):
        pblock = self.get_parameter_block(JustOneP)
        self.assertEqual(pblock, ['P1'])
        
    def test_skip_p2(self):
        pblock = self.get_parameter_block(SkipP2)
        self.assertEqual(pblock, ['P1', False,'P3'])

    def test_out_of_order_parameters_unnamed(self):
        self.assertRaises(SyntaxError, lambda : self.get_parameter_block(ReversedParameters))

    def test_named_parameter(self):
        pblock = self.get_parameter_block(OneNamedParameter)
        self.assertEqual(pblock, ['named_parameter_one'])
        
    def test_mixed_names(self):
        pblock = self.get_parameter_block(MixedNaming)
        self.assertEqual(pblock, ['P1', 'named',False, 'P4', 'named2'])
            


unittest.main(argv=[''], verbosity=3, exit=False)


test_known_dict (__main__.TestB62Encoder.test_known_dict)
Note: Naked dicts are encoded as blueprints, not behaviors by default. ... ok
test_known_list (__main__.TestB62Encoder.test_known_list) ... ok
test_known_string (__main__.TestB62Encoder.test_known_string) ... ok
test_roundtrip (__main__.TestB62Encoder.test_roundtrip) ... ok
test_just_p1 (__main__.TestParameterNaming.test_just_p1) ... ok
test_mixed_names (__main__.TestParameterNaming.test_mixed_names) ... ok
test_named_parameter (__main__.TestParameterNaming.test_named_parameter) ... ok
test_out_of_order_parameters_unnamed (__main__.TestParameterNaming.test_out_of_order_parameters_unnamed) ... ok
test_skip_p2 (__main__.TestParameterNaming.test_skip_p2) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.019s

OK


[['in', 'To', None, 'coord_num'], ['in', 'Num', None, 'coord_num'], ['out', 'Result']]
[['in', 'To', None, 'coord_num'], ['in', 'Num', None, 'coord_num'], ['out', 'Result']]


In [4]:
def While_Simple():
    while CompareNumber(A,B):
        A+=1
        
def While_with_BeforeAfter():
    F=1
    while CompareNumber(A,B):
        A+=1
    F=2

def While_Nested():
    while CompareNumber(A,B):
        while CompareNumber(A,C):
            A+=1


def While_Nested_BeforeAfter():
    F=1
    while CompareNumber(A,B):
        while CompareNumber(A,C):
            A+=1
    F=2
    
def While_CompoundTest():
    while CompareNumber(A+1,B+2):
        A+=1

def While_Pass():
    while CompareNumber(A,B):
        pass

class Test_WhileLoops(unittest.TestCase):
    def test_While_Simple(self):
        tree, flow = getFlow(While_Simple)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[0]['next']['exit'], -1)
        self.assertEqual(flow[1]['next']['next'], 0)

    def test_While_BeforeAfter(self):
        tree, flow = getFlow(While_with_BeforeAfter)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[1]['next']['exit'], 3)
        self.assertEqual(flow[1]['next']['next'], 2)
        self.assertEqual(flow[2]['next']['next'], 1)
        self.assertEqual(flow[3]['next']['next'], -1)

    def test_While_Nested(self):
        tree, flow = getFlow(While_Nested)
        #for key in flow.keys(): print(key, flow[key]['next'])
        
        self.assertEqual(flow[0]['next']['exit'], -1)
        self.assertEqual(flow[0]['next']['next'], 1)  
        self.assertEqual(flow[1]['next']['exit'], 0)
        self.assertEqual(flow[1]['next']['next'], 2)
        self.assertEqual(flow[2]['next']['next'], 1)

    def test_While_Nested_BeforeAfter(self):
        tree, flow = getFlow(While_Nested_BeforeAfter)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[1]['next']['exit'], 4)
        self.assertEqual(flow[1]['next']['next'], 2)  
        self.assertEqual(flow[2]['next']['exit'], 1)
        self.assertEqual(flow[2]['next']['next'], 3)
        self.assertEqual(flow[3]['next']['next'], 2)
        self.assertEqual(flow[4]['next']['next'], -1)

    def test_While_CompoundTest(self):
        tree, flow = getFlow(While_CompoundTest)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[1]['next']['next'], 2)
        self.assertEqual(flow[2]['next']['next'], 3)  
        self.assertEqual(flow[2]['next']['exit'], -1)  
        self.assertEqual(flow[3]['next']['next'], 0)

    def test_While_Pass(self):
        tree, flow = getFlow(While_Pass)
        for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 0)
        self.assertEqual(flow[0]['next']['exit'], -1)






unittest.main(argv=[''], verbosity=3, exit=False)

test_known_dict (__main__.TestB62Encoder.test_known_dict)
Note: Naked dicts are encoded as blueprints, not behaviors by default. ... ok
test_known_list (__main__.TestB62Encoder.test_known_list) ... ok
test_known_string (__main__.TestB62Encoder.test_known_string) ... ok
test_roundtrip (__main__.TestB62Encoder.test_roundtrip) ... ok
test_just_p1 (__main__.TestParameterNaming.test_just_p1) ... ok
test_mixed_names (__main__.TestParameterNaming.test_mixed_names) ... ok
test_named_parameter (__main__.TestParameterNaming.test_named_parameter) ... ok
test_out_of_order_parameters_unnamed (__main__.TestParameterNaming.test_out_of_order_parameters_unnamed) ... ok
test_skip_p2 (__main__.TestParameterNaming.test_skip_p2) ... ok
test_While_BeforeAfter (__main__.Test_WhileLoops.test_While_BeforeAfter) ... ok
test_While_CompoundTest (__main__.Test_WhileLoops.test_While_CompoundTest) ... ok
test_While_Nested (__main__.Test_WhileLoops.test_While_Nested) ... ok
test_While_Nested_BeforeAfter (__main__.Tes

[['in', 'To', None, 'coord_num'], ['in', 'Num', None, 'coord_num'], ['out', 'Result']]
[['in', 'To', None, 'coord_num'], ['in', 'Num', None, 'coord_num'], ['out', 'Result']]
0 {'next': 0, 'exit': -1}


In [5]:

def If_Simple(P1, P2, P3):
    if CompareNumber(P1,P2):
        P3 = P1
        
def If_Else(P1, P2, P3):
    if CompareNumber(P1,P2):    #0 1, 2
        P3 = P1                 #1 -1
    else:
        P3 = P1                 #2 -1

def If_Elif(P1, P2, P3):
    if CompareNumber(P1,P2):    #0 1, 2
        P3 = P1                 #1 -1
    elif "Greater":
        P3 = P1                 #2 -1

def If_Nested(P1, P2, P3):
    if CompareNumber(P1):
        if CompareNumber(P1):
            if CompareNumber(P1):
                A=1

def If_FourWay(P1,P2,P3,P4):
    if CompareNumber(P1):       #0   1, 4
        if CompareNumber(P2):   #1   2, 3
            A=1                 #2   -1
        else:
            B=1                 #3   -1
    else:
        if CompareNumber(P2):   #4  5, 6
            C=1                 #5  -1
        else:
            D=1                 #6  -1

                
class TestIfElseElif(unittest.TestCase):    
    def test_If_Simple(self):
        tree, flow = getFlow(If_Simple)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[0]['next']['exit'], -1)
        self.assertEqual(flow[1]['next']['next'], -1)
    
    def test_If_Else(self):
        tree, flow = getFlow(If_Else)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[0]['next']['exit'], 2)
        self.assertEqual(flow[1]['next']['next'], -1)
        self.assertEqual(flow[2]['next']['next'], -1)    
        
    def test_If_Elif(self):
        tree, flow = getFlow(If_Elif)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[0]['next']['Greater'], 2)
        self.assertEqual(flow[1]['next']['next'], -1)
        self.assertEqual(flow[2]['next']['next'], -1)

    def test_If_Nested(self):
        tree, flow = getFlow(If_Nested)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[0]['next']['exit'], -1)
        self.assertEqual(flow[1]['next']['next'], 2)
        self.assertEqual(flow[1]['next']['exit'], -1)
        self.assertEqual(flow[2]['next']['next'], 3)
        self.assertEqual(flow[2]['next']['exit'], -1)
        self.assertEqual(flow[3]['next']['next'], -1)   
        
    def test_If_FourWay(self):
        tree, flow = getFlow(If_FourWay)
        #for key in flow.keys(): print(key, flow[key]['next'])

        self.assertEqual(flow[0]['next']['next'], 1)
        self.assertEqual(flow[0]['next']['exit'], 4)
        self.assertEqual(flow[1]['next']['next'], 2)
        self.assertEqual(flow[1]['next']['exit'], 3)
        self.assertEqual(flow[4]['next']['next'], 5)
        self.assertEqual(flow[4]['next']['exit'], 6)

        self.assertEqual(flow[2]['next']['next'], -1)
        self.assertEqual(flow[3]['next']['next'], -1)
        self.assertEqual(flow[5]['next']['next'], -1)
        self.assertEqual(flow[6]['next']['next'], -1)



unittest.main(argv=[''], verbosity=3, exit=False)


test_known_dict (__main__.TestB62Encoder.test_known_dict)
Note: Naked dicts are encoded as blueprints, not behaviors by default. ... ok
test_known_list (__main__.TestB62Encoder.test_known_list) ... ok
test_known_string (__main__.TestB62Encoder.test_known_string) ... ok
test_roundtrip (__main__.TestB62Encoder.test_roundtrip) ... ok
test_If_Elif (__main__.TestIfElseElif.test_If_Elif) ... ok
test_If_Else (__main__.TestIfElseElif.test_If_Else) ... ok
test_If_FourWay (__main__.TestIfElseElif.test_If_FourWay) ... ok
test_If_Nested (__main__.TestIfElseElif.test_If_Nested) ... ok
test_If_Simple (__main__.TestIfElseElif.test_If_Simple) ... ok
test_just_p1 (__main__.TestParameterNaming.test_just_p1) ... ok


[['in', 'To', None, 'coord_num'], ['in', 'Num', None, 'coord_num'], ['out', 'Result']]


test_mixed_names (__main__.TestParameterNaming.test_mixed_names) ... ok
test_named_parameter (__main__.TestParameterNaming.test_named_parameter) ... ok
test_out_of_order_parameters_unnamed (__main__.TestParameterNaming.test_out_of_order_parameters_unnamed) ... ok
test_skip_p2 (__main__.TestParameterNaming.test_skip_p2) ... ok
test_While_BeforeAfter (__main__.Test_WhileLoops.test_While_BeforeAfter) ... ok
test_While_CompoundTest (__main__.Test_WhileLoops.test_While_CompoundTest) ... ok
test_While_Nested (__main__.Test_WhileLoops.test_While_Nested) ... ok
test_While_Nested_BeforeAfter (__main__.Test_WhileLoops.test_While_Nested_BeforeAfter) ... ok
test_While_Pass (__main__.Test_WhileLoops.test_While_Pass) ... ok
test_While_Simple (__main__.Test_WhileLoops.test_While_Simple) ... ok

----------------------------------------------------------------------
Ran 20 tests in 0.077s

OK


[['in', 'To', None, 'coord_num'], ['in', 'Num', None, 'coord_num'], ['out', 'Result']]
0 {'next': 0, 'exit': -1}


In [6]:
def NestedWhile():
    while CompareNumber(A,B):
        while CompareNumber(C,D):
            E=F
tree = ast.parse(inspect.getsource(NestedWhile), type_comments=False)
tree = replace_binops_with_functions(tree)
tree = flatten_calls(tree)
tree = convert_to_ds_call(tree)
tree = label_frames_vars(tree)
flow_control(tree)

0

In [7]:
import ast
import json


def export_json(tree, pretty_print=False):
    if not isinstance(tree, ast.AST):
        raise ValueError(
            "The argument of export_json(..) must be of type AST, not '{}'".format(
                type(tree)
            )
        )
    return json.dumps(
        export_dict(tree),
        indent=4 if pretty_print else None,
        sort_keys=True,
        separators=(",", ": ") if pretty_print else (",", ":")
    )


def export_dict(tree):
    if not isinstance(tree, ast.AST):
        raise ValueError(
            "The argument of export_dict(..) must be of type AST, not '{}'".format(
                type(tree)
            )
        )
    return DictExportVisitor().visit(tree)


class DictExportVisitor:
    ast_type_field = "ast_type"

    def visit(self, node):
        node_type = node.__class__.__name__
        meth = getattr(self, "visit_" + node_type, self.default_visit)
        return meth(node)

    def default_visit(self, node):
        node_type = node.__class__.__name__
        # Add node type
        args = {
            self.ast_type_field: node_type
        }
        # Visit fields
        for field in node._fields:
            assert field != self.ast_type_field
            meth = getattr(
                self, "visit_field_" + node_type + "_" + field,
                self.default_visit_field
            )
            args[field] = meth(getattr(node, field))
        # Visit attributes
        '''
        for attr in node._attributes:
            assert attr != self.ast_type_field
            meth = getattr(
                self, "visit_attribute_" + node_type + "_" + attr,
                self.default_visit_field
            )
            # Use None as default when lineno/col_offset are not set
            args[attr] = meth(getattr(node, attr, None))
            '''
        return args

    def default_visit_field(self, val):
        if isinstance(val, ast.AST):
            return self.visit(val)
        if isinstance(val, (list, tuple)):
            return [self.visit(x) for x in val]
        return val

    # Special visitors

    def visit_str(self, val):
        return str(val)

    def visit_Bytes(self, val):
        return str(val.s)

    def visit_NoneType(self, val):
        del val  # Unused
        return None

    def visit_field_NameConstant_value(self, val):
        return str(val)

    def visit_field_Num_n(self, val):
        if isinstance(val, int):
            return {
                self.ast_type_field: "int",
                "n": val,
                # JavaScript integers are limited to 2**53 - 1 bits,
                # so we add a string representation of the integer
                "n_str": str(val),
            }
        if isinstance(val, float):
            return {
                self.ast_type_field: "float",
                "n": val
            }
        if isinstance(val, complex):
            return {
                self.ast_type_field: "complex",
                "n": val.real,
                "i": val.imag
            }